In [1]:
from selenium import webdriver


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

browser = webdriver.Edge()
browser.get('https://www.google.com/')


In [2]:
import requests
import json
#點選「更多評論」後的網址
link = "https://www.booking.com/reviewlist.zh-tw.html?aid=309654&label=hotels-chinese-zh-xt-not-tw-3E%2AoDaZgyjzN82axX_iA0wS168630596047%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-320553767261%3Alp9051484%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YcsZ-Id2vkzIfTmYhvC5HOg&sid=612b1d9bc12dd29048d367a325a1adf8&srpvid=6379249454980051&;cc1=tw&pagename=caesar-park-kenting&r_lang=&review_topic_category_id=&type=total&score=&sort=f_recent_desc&time_of_year=&dist=1&offset=&rows=10&rurl=&text=&translate=&_=1687267902319"
browser.get(link)
time.sleep(2)

In [5]:
#0620 :可以抓到標題、正評論、負評，需要寫判斷式來辨識正評、負評
#還要寫程式來讓selenium可以換頁


review_blocks = browser.find_elements(By.CLASS_NAME, "c-review-block__right")


# 逐一處理每個評論區塊
for block in review_blocks:
    # 找到標題元素
    title_element = block.find_element(By.CLASS_NAME, "c-review-block__title")
    title = title_element.text
    print(f"title:  {title}")

    # 找到評論元素
    reviews = block.find_elements(By.CLASS_NAME, "c-review__body")
    for review in reviews:
        
        review_text = review.text
        print(f"review: {review_text}")

    print("------------------")

title:  位置雖佳.但價格不便宜.飯店不少小細節不貼心.沒有高級飯店的水準.應該不會再去了.
review: 位置佳.早餐多樣豐富好吃.服務人員態度親切積極.
review: 網路差.很差.非常差....戶外泡澡池材質很粗造.踩踏下去很不舒服.若是跪著.膝蓋很痛..
------------------
title:  傑出
review: 客人沒有留下任何評語。
------------------
title:  傑出
review: 客人沒有留下任何評語。
------------------
title:  好極了
review: 客人沒有留下任何評語。
------------------
title:  傑出
review: 客人沒有留下任何評語。
------------------
title:  非常好
review: 房務員來加裝小朋友護欄，很親切、用心。 去沙灘可以登記使用沙灘傘跟躺椅，一房一組。
review: 早餐選第一時段，興許是入住人數眾多，桌數不夠/整理不及，導致住客在外排隊等待入席，但不巧用餐時段快結束，吃得很匆忙。
------------------
title:  好極了
review: 房間乾淨整潔，自然光線充足明亮，後門直通飯店泳池，非常方便，室外的浴池夠大，給小朋友玩水也很適合，房間的戶外區有僞芭厘島風，停車用餐皆便利，過馬路就是海灘。
------------------
title:  浴室拉門有點問題,無法拉動門,有反應給房務,但尚未維修
review: 客人沒有留下任何評語。
------------------
title:  傑出
review: 客人沒有留下任何評語。
------------------
title:  好極了
review: 客人沒有留下任何評語。
------------------
